In [1]:
# import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.preprocessing import StandardScaler

# Data Preparation

In [2]:
class CustomDataset(Dataset):
    
    def __init__(self, filepath) -> None:
        
        # load csv data
        data = pd.read_csv(filepath, header=None)
        X = data.iloc[:, :-1].values
        y = data.iloc[:, -1].values
        
        # feature scaling
        sc = StandardScaler()
        X = sc.fit_transform(X)
        
        # convert to tensors
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index].view(-1)

In [3]:
dataset = CustomDataset('../data/train_all_0.csv')

# create data indices for train val split
data_size = len(dataset)
indices = list(range(data_size))
split = int(np.floor(0.2 * data_size))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# create data loader
train_loader = DataLoader(dataset, batch_size=16, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=16, sampler=val_sampler)

In [18]:
def binary_acc(y_pred, y_true):
    y_pred_tag = torch.round(torch.sigmoid(y_pred)).view(-1)
    correct_result_sum = (y_pred_tag == y_true).sum().float()
    acc = correct_result_sum / y_true.shape[0]
    acc = torch.round(acc)
    return acc

In [19]:
# create training pipeline
def train(train_loader=train_loader, val_loader=val_loader, model=None, epochs=None, criterion=None, optimizer=None, device=None):
    for epoch in range(epochs):
        
        model.train()
        train_loss = []
        train_accs = []
        
        for batch in train_loader:
            
            x, y = batch
            
            y_pred = model(x.to(device))
            loss = criterion(y_pred, y.to(device).to(torch.float))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc = binary_acc(y_pred, y.view(-1))
            
            train_loss.append(loss.item())
            train_accs.append(acc)
        
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)
        
        model.eval()

        valid_loss = []
        valid_accs = []
        
        for batch in val_loader:
            x, y = batch
            
            with torch.no_grad():
                y_pred = model(x.to(device))
                
                acc = binary_acc(y_pred, y.view(-1))
                
                valid_loss.append(loss.item())
                valid_accs.append(acc)
        
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)
        
        print(f'[ {epoch+1}/{epochs} ] | train_loss = {train_loss:.5f}, train_acc = {train_acc:.5f}, val_loss = {valid_loss:.5f}, val_acc = {valid_acc:.5f}')

# KNN

In [87]:
# Not necessary to use pytorch

class KNN:
    def __init__(self, train_loader, val_loader, k, weight='uniform'):
        
        # initialize data
        self.X_train = train_loader.dataset.X
        self.y_train = train_loader.dataset.y
        
        self.X_val = val_loader.dataset.X
        self.y_val = val_loader.dataset.y
        
        self.n_neighbors = k
        self.weight = weight
        
        self.n_classes = len(set((train_loader.dataset.y).numpy()))
    
    def euclidean_dist(self, a, b):
        return torch.sqrt(torch.sum((a - b)**2, dim=0))
    
    def k_neighbors(self, return_dist=False):
        
        dist = []
        neigh_ind = []
        
        point_dist = [self.euclidean_dist(x_val, self.X_train) for x_val in self.X_val]
        
        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh,
                                  key=lambda x: x[1])[:self.n_neighbors]
            
            ind_list = [tup[0] for tup in sorted_neigh]
            dist_list = [tup[1] for tup in sorted_neigh]
            
            dist.append(dist_list)
            neigh_ind.append(ind_list)
            
        if return_dist:
            return np.array(dist), np.array(neigh_ind)
        
        return np.array(neigh_ind)
    
    def predict(self):
        
        if self.weight == 'uniform':
            neighbors = self.k_neighbors(False)
            y_pred = np.array([
                np.argmax(np.bincount(self.y_train[neighbor]))
                for neighbor in neighbors
            ])
            
            return y_pred
        
        if self.weight == 'distance':
            dist, neigh_ind = self.k_neighbors(True)
            
            inv_dist = 1 / dist
            mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis]
            
            prob = []
            
            for i, row in enumerate(mean_inv_dist):
                row_pred = self.y_train[neigh_ind[i]]
                
                for k in range(self.n_classes):
                    indices = np.where(row_pred == k)
                    prob_ind = np.sum(row[indices])
                    prob.append(np.array(prob_ind))
            
            predict_prob = np.array(prob).reshape(self.X_val.shape[0], self.n_classes)
            y_pred = np.array([np.argmax(item) for item in predict_prob])
            
            return y_pred
    
    def score(self):
        y_pred = self.predict()
        return float(sum(y_pred == np.array(self.y_val))) / float(len(self.y_val))

In [38]:
knn_classifier = KNN(train_loader=train_loader, val_loader=val_loader, k=2, weight='distance')
accuracy = knn_classifier.score()
print(accuracy)

0.3958333333333333


# SVM

In [126]:
class hinge_loss(nn.modules.Module):    
    def __init__(self):
        super(hinge_loss,self).__init__()
    def forward(self, outputs, labels):
        return torch.sum(torch.clamp(1 - outputs.t()*labels, min=0))/16

In [128]:
epochs = 10
device = torch.device('cpu')
model = nn.Linear(12, 1).to(device)
criterion = hinge_loss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
train(train_loader, val_loader, model, epochs, criterion, optimizer, device)

[ 1/10 ] | train_loss = 16.82097, train_acc = 0.29167, val_loss = 18.55934, val_acc = 0.00000
[ 2/10 ] | train_loss = 16.50279, train_acc = 0.16667, val_loss = 18.08690, val_acc = 0.00000
[ 3/10 ] | train_loss = 16.15704, train_acc = 0.37500, val_loss = 18.23576, val_acc = 0.33333
[ 4/10 ] | train_loss = 15.87426, train_acc = 0.20833, val_loss = 15.56667, val_acc = 0.33333
[ 5/10 ] | train_loss = 15.48034, train_acc = 0.41667, val_loss = 15.83921, val_acc = 0.16667
[ 6/10 ] | train_loss = 15.17979, train_acc = 0.37500, val_loss = 16.32392, val_acc = 0.33333
[ 7/10 ] | train_loss = 14.89297, train_acc = 0.50000, val_loss = 11.72066, val_acc = 0.33333
[ 8/10 ] | train_loss = 14.55318, train_acc = 0.41667, val_loss = 16.24359, val_acc = 0.16667
[ 9/10 ] | train_loss = 14.32997, train_acc = 0.45833, val_loss = 13.47911, val_acc = 0.33333
[ 10/10 ] | train_loss = 13.98332, train_acc = 0.54167, val_loss = 13.44308, val_acc = 0.66667


# Softmax Classifier

In [ ]:
class CustomDataset(Dataset):
    
    def __init__(self, filepath) -> None:
        
        # load csv data
        data = pd.read_csv(filepath, header=None)
        X = data.iloc[:, :-1].values
        y = data.iloc[:, -1].values
        
        # feature scaling
        sc = StandardScaler()
        X = sc.fit_transform(X)
        
        # convert to tensors
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self) -> int:
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index].view(-1)

In [ ]:
class SoftmaxClassifier(nn.Module):
    def __init__(self):
        super(SoftmaxClassifier, self).__init__()
        self.layer_1 = nn.Linear(12, 2) 
        
    def forward(self, inputs):
        x = self.layer_1(inputs)
        
        return x

In [124]:
epochs = 10
device = torch.device('cpu')
model = SoftmaxClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
train(train_loader, val_loader, model, epochs, criterion, optimizer)

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

# Two Layer NN

In [21]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        self.layer_1 = nn.Linear(12, 2) 
        self.layer_out = nn.Linear(2, 1)
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.layer_out(x)
        
        return x

In [22]:
epochs = 10
device = torch.device('cpu')
model = BinaryClassification().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
train(train_loader, val_loader, model, epochs, criterion, optimizer)

[ 1/10 ] | train_loss = 0.74624, train_acc = 0.04167, val_loss = 0.73345, val_acc = 0.00000
[ 2/10 ] | train_loss = 0.71151, train_acc = 0.16667, val_loss = 0.64198, val_acc = 0.33333
[ 3/10 ] | train_loss = 0.67679, train_acc = 0.58333, val_loss = 0.67344, val_acc = 0.50000
[ 4/10 ] | train_loss = 0.64406, train_acc = 0.75000, val_loss = 0.64069, val_acc = 1.00000
[ 5/10 ] | train_loss = 0.61370, train_acc = 0.91667, val_loss = 0.61045, val_acc = 1.00000
[ 6/10 ] | train_loss = 0.58578, train_acc = 0.95833, val_loss = 0.52450, val_acc = 1.00000
[ 7/10 ] | train_loss = 0.56032, train_acc = 1.00000, val_loss = 0.56835, val_acc = 1.00000
[ 8/10 ] | train_loss = 0.53820, train_acc = 1.00000, val_loss = 0.62928, val_acc = 1.00000
[ 9/10 ] | train_loss = 0.51798, train_acc = 1.00000, val_loss = 0.57216, val_acc = 1.00000
[ 10/10 ] | train_loss = 0.50062, train_acc = 1.00000, val_loss = 0.67178, val_acc = 1.00000
